In [1]:
# ------ 요청사항 ------

# 0. 파일 경로 
# file_path = r"D:\pythondata\★_재배치_회수.csv"
# file_path = r"D:\pythondata\★_재배치_배송.csv"
# 1. 먼저, 두 파일을 하나의 파일로 합친 후 
# 2. 회수 파일과 배송 파일을 구분하기 위해서 "구분"이라는 이름의 필드를 마지막 컬럼으로 생성해서 적어두면 좋겠어요.
# 3. 원본데이터 필드는 "자전거번호,대여소,대여소명,날짜,시간"이고, "날짜+시간"이라는 필드를 만들어 주세요.
# 4. "대여소, 날짜, 시간"으로 정렬(sorting)하고 시작하는데,
# 5. No라는 컬럼을 가장 왼쪽에 생성하고, 정렬 순서(단순 정렬 개념)에 따라서 제일 윗줄부터 1로 시작하면 좋겠어요.
# 6. 이 파일을 "5-1_회수배송_데이터(1)"라는 이름으로 csv파일을 저장해 주세요.
# 7. 정리하면, "두 개 파일(★_재배치_회수.csv, ★_재배치_배송.csv)을 합쳐서 "구분"이라는 이름의 필드와 
#    "No"라는 이름의 필드, "날짜+시간" 필드를 새로 만든다. 
#    "대여소"가 낮은 순서(102번부터 시작)이면서 "날짜+시간" 필드가 
#    빠른 순서(오름차순 정렬, 2023년 1월 1일 0시 0분 0초부터)로 정렬. 정렬된 순서에 따라서 "No"의 번호 넣기" 입니다!

In [2]:
import pandas as pd

# 0. 파일 경로
path_retrieve = r"D:\pythondata\★_재배치_회수.csv"
path_delivery = r"D:\pythondata\★_재배치_배송.csv"

# 1. 파일 불러오기 (탭 구분자 + 인코딩 처리 + 대여소 문자 처리)
df_retrieve = pd.read_csv(path_retrieve, encoding='utf-8-sig', sep="\t", dtype={'대여소': str})
df_delivery = pd.read_csv(path_delivery, encoding='utf-8-sig', sep="\t", dtype={'대여소': str})

# 2. '구분' 필드 추가 (일단 추가는 지금 하고, 나중에 마지막 컬럼으로 이동)
df_retrieve['구분'] = '회수'
df_delivery['구분'] = '배송'

# 3. 병합
df = pd.concat([df_retrieve, df_delivery], ignore_index=True)

# 4. '날짜+시간' 필드 생성
df['날짜+시간'] = pd.to_datetime(df['날짜'] + ' ' + df['시간'])

# 5. 정렬 기준: ① 대여소 번호를 숫자로 변환해서 정렬 (102가 위로) ② 날짜+시간 오름차순
df['대여소_정렬용'] = pd.to_numeric(df['대여소'], errors='coerce')  # 숫자로 변환 불가능한 것은 NaN 처리
df = df.sort_values(by=['대여소_정렬용', '날짜+시간']).reset_index(drop=True)
df.drop(columns='대여소_정렬용', inplace=True)  # 임시 정렬 컬럼 제거

# 6. No 필드 추가 (가장 왼쪽에 삽입)
df.insert(0, 'No', df.index + 1)

# 7. 컬럼 순서 정리: '구분'은 마지막으로 이동
cols = [col for col in df.columns if col != '구분'] + ['구분']
df = df[cols]

# 8. 저장 및 완료 메시지
output_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 5-1 파일 저장 완료!", output_path)

✅ 5-1 파일 저장 완료! D:\pythondata\5-1_회수배송_데이터(1).csv


In [3]:
# ------ 요청사항 ------

# 1. file_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"를 다시 읽고,  
# 2. 추가해서 만들 필드는 "방문코드,순번,flag,자전거대수,first data&time,last data&time" 6개예요. 
# 3. "5-2_회수배송_데이터(2).csv"파일에서 가질 필드는 총 14개이고, 
#    "No,자전거번호,대여소,대여소명,날짜,시간,날짜+시간,구분,방문코드,순번,flag,자전거대수,first data&time,last data&time"으로 순서를 지정해 주세요. 
#    "5-1_회수배송_데이터(1).csv"에 있는 데이터(No,자전거번호,대여소,대여소명,날짜,시간,날짜+시간)는 그대로 두고, 
#    방문코드부터 last data&time까지 6개 필드를 새로 만드는 거예요. 
# 4. "대여소"(102번 대여소가 가장 위에 와야 함)별로 "자전거번호"의 "날짜+시간"의 간격이 "30분"이상일 때 "방문코드"가 변경되는 것이 원칙으로 할 거예요. 
# 5. "방문코드"의 시작 번호는 "A0000001"로 시작해야 하구요. "순번"은 "방문코드"에 따라서 1부터 시작해요. 
# 6. "방문코드"를 세어(count) 준 숫자가 "자전거대수" 필드에 들어가야 해요. 
# 7. "flag" 칼럼은 동일한 "방문코드"를 가지고 있을 때, 동일한 "자전거번호"가 30분 이내에 위아래 행에 위치하면 아래 행에만 "가짜"라고 적으면 돼요. 
# 8. "first data&time"과 "last data&time"은 동일한 "방문코드"의 "순번" "1"이 "first data&time"에 위치하면 되고, 
#    해당 "방문코드"의 "마지막 순번"이 "last data&time"에 위치하면 돼요.  
# 9. "flag" 칼럼의 "가짜"라는 단어때문에 코딩이 꼬여서 모든 칸이 채워지지 않았으면 좋겠어요. 해당 순번이 있는 행(row)에만 위치하면 돼요.  
# 10. "flag" 칼럼에 "가짜"라는 단어가 있어도, 동일 "방문코드"의 마지막 "순번"에서 "날짜+시간" 중 마지막이요!! 
# 11. 이 파일의 이름은 "5-2_회수배송_데이터(2)"라고 지정해서 저장해 주세요.

In [4]:
import pandas as pd
from datetime import timedelta

# 📌 1. 파일 불러오기
file_path = r"D:\pythondata\5-1_회수배송_데이터(1).csv"
df = pd.read_csv(file_path, encoding='utf-8-sig', dtype={
    '대여소': str,
    '대여소명': str,
    '자전거번호': str
})
df['날짜+시간'] = pd.to_datetime(df['날짜+시간'])

In [5]:
# ✅ 정확한 방문코드 생성 기준 (정리)
# 🔹 방문코드 분기 조건:
# 같은 대여소 기준으로,
# "날짜+시간"이 30분 이상 차이 날 경우, 새로운 방문코드 부여
# 자전거번호는 묶음 내부에서만 구분을 위한 정보 (flag 체크 등에 사용)
# 🔹 방문코드 관련 요구사항:
# 방문코드 시작은 A0000001
# 같은 방문코드 내에서는 순번은 1부터 시작
# 같은 방문코드 내 자전거대수는 unique한 자전거번호 수
# 30분 이내에 동일 자전거가 다시 등장하면 아래행에만 flag = "가짜"
# 방문코드 내 첫 row는 first data&time, 마지막 row는 last data&time

# ✅ 5-2 전체 코드 요약
# 이 코드는 다음을 수행합니다:
# 5-1_회수배송_데이터(1).csv 읽기
# 같은 대여소 기준으로 정렬하고, 30분 이상 간격 발생 시 방문코드 변경
# 방문코드 시작은 "A0000001"
# 순번, 자전거대수, flag, first/last data&time 모두 포함
# 최종 열 순서대로 저장

In [6]:
# 📌 2. 방문코드 및 순번 생성 (A0000001부터 시작)
df['flag'] = ''
df['자전거대수'] = 0
df['first data&time'] = ''
df['last data&time'] = ''

df['원래순서'] = df.index
df = df.sort_values(by=['대여소', '날짜+시간']).reset_index(drop=True)

visit_index = 1
visit_code = [f"A{visit_index:07d}"]  # A0000001 시작

for i in range(1, len(df)):
    time_diff = df.loc[i, "날짜+시간"] - df.loc[i - 1, "날짜+시간"]
    location_changed = df.loc[i, "대여소"] != df.loc[i - 1, "대여소"]
    if time_diff.total_seconds() > 1800 or location_changed:
        visit_index += 1
    visit_code.append(f"A{visit_index:07d}")

df["방문코드"] = visit_code
df["순번"] = df.groupby("방문코드").cumcount() + 1

df = df.sort_values(by='원래순서').reset_index(drop=True)
df.drop(columns='원래순서', inplace=True)

In [7]:
# 📌 3. flag 생성
for i in range(1, len(df)):
    curr = df.iloc[i]
    prev = df.iloc[i - 1]
    if (
        curr['자전거번호'] == prev['자전거번호'] and
        curr['방문코드'] == prev['방문코드'] and
        (curr['날짜+시간'] - prev['날짜+시간']) <= timedelta(minutes=30)
    ):
        df.at[i, 'flag'] = '가짜'

In [8]:
# 📌 4. first data&time, last data&time
for code, group in df.groupby('방문코드'):
    sorted_group = group.sort_values(by='순번')
    first_idx = sorted_group.index[0]
    last_idx = sorted_group.index[-1]
    df.at[first_idx, 'first data&time'] = sorted_group.iloc[0]['날짜+시간']
    df.at[last_idx, 'last data&time'] = sorted_group.iloc[-1]['날짜+시간']

In [9]:
# 📌 5. 자전거대수 계산
df['자전거대수'] = df.groupby('방문코드')['자전거번호'].transform('nunique')

In [10]:
# 📌 6. 컬럼 순서 정리
final_columns = ['No', '자전거번호', '대여소', '대여소명', '날짜', '시간', '날짜+시간',
                 '구분', '방문코드', '순번', 'flag', '자전거대수',
                 'first data&time', 'last data&time']
df = df[final_columns]

In [11]:
# 📌 7. 저장
output_path = r"D:\pythondata\5-2_회수배송_데이터(2).csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 5-2 파일 저장 완료!")
print("📁 저장 위치:", output_path)

✅ 5-2 파일 저장 완료!
📁 저장 위치: D:\pythondata\5-2_회수배송_데이터(2).csv
